In [1]:
import syft as sy

assigning id:<UID:d03ff337-910b-4565-a577-d5dd437fa0b8> type:<class 'torch.Tensor'>


### Test 1: Send a Message from a VM Client to a VM

In [2]:
bob_vm = sy.VirtualMachine(name="Bob")
bob_vm_client = bob_vm.get_client()

In [3]:
assert bob_vm.device is None

In [4]:
bob_vm_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

the message is for me!!!
VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>


### Test 2: Send a message from a Device Client to a Device

In [5]:
bob_phone = sy.Device(name="Bob's iPhone")
bob_phone_client = bob_phone.get_client()

In [6]:
msg = sy.ReprMessage(address=bob_phone_client)

In [7]:
bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_phone_client))

the message is for me!!!
Device:Bob's iPhone:<UID:cbcfd989-8878-4b3a-a2d2-f3c4514b889e>


### Test 2.1: Register a VM on a Device

In [8]:
bob_phone_client.register(client=bob_vm_client)

the message is for me!!!
the message is for me!!!
Updating to Device:Bob's iPhone:<UID:cbcfd989-8878-4b3a-a2d2-f3c4514b889e> on note VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
child nodes of:Bob


In [9]:
bob_phone.store

{<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>: <Storable:<VirtualMachineClient id:Bob>>}

### Test 2.2 Test that registration updated "address" variable of VM

In [10]:
assert bob_vm.device is not None

In [11]:
assert bob_vm_client.device is not None

### Test 3: Send a message from a Device Client to a VM

In [12]:
bob_phone_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm_client))

the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is for me!!!
VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>


### Test 4: Send a message from a Domain Client to a VM

In [13]:
import syft as sy

In [14]:
bob_domain = sy.Domain(name="Bob's Domain")
bob_domain_client = bob_domain.get_client()

In [15]:
bob_domain_client.register(client=bob_phone_client)

the message is for me!!!
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:5c205b80-bbba-449c-9b3b-4d79c9c44512> on note Device:Bob's iPhone:<UID:cbcfd989-8878-4b3a-a2d2-f3c4514b889e>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Domain:Bob's Domain:<UID:5c205b80-bbba-449c-9b3b-4d79c9c44512> on note VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
child nodes of:Bob


In [16]:
bob_domain_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is for me!!!
VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>


### Test 5: Send a message from Network Client to a VM

In [17]:
bob_network = sy.Network(name="Bob's Network")
bob_network_client = bob_network.get_client()

In [18]:
bob_network_client.register(client=bob_domain_client)

the message is for me!!!
the message is for me!!!
Updating to Network:Bob's Network:<UID:36ec6f58-98fe-4016-9c6e-1e98bebf10cb> on note Domain:Bob's Domain:<UID:5c205b80-bbba-449c-9b3b-4d79c9c44512>
child nodes of:Bob's Domain
	Bob's iPhone
the message is for me!!!
Updating to Network:Bob's Network:<UID:36ec6f58-98fe-4016-9c6e-1e98bebf10cb> on note Device:Bob's iPhone:<UID:cbcfd989-8878-4b3a-a2d2-f3c4514b889e>
child nodes of:Bob's iPhone
	Bob
the message is for me!!!
Updating to Network:Bob's Network:<UID:36ec6f58-98fe-4016-9c6e-1e98bebf10cb> on note VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
child nodes of:Bob


In [19]:
bob_network_client.send_immediate_msg_without_reply(msg=sy.ReprMessage(address=bob_vm))

the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is not for me...
<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>
the message is for me!!!
VirtualMachine:Bob:<UID:4896c1ef-8ec5-46a7-8bd2-ee8337787a9e>


### Test 5: Use VM Client wtih Torch Tensor

In [1]:
import syft as sy
alice = sy.VirtualMachine(name="alice")
alice_client = alice.get_client()

assigning id:<UID:79c78a01-7628-4e37-83e3-648f3381c907> type:<class 'torch.Tensor'>


In [2]:
import torch as th

In [3]:
x = th.tensor([1,2,3,4])

blob = x.serialize()

x2 = sy.deserialize(blob=blob)

assert (x == x2).all()

assigning id:<UID:2582de0b-5dc8-4725-a264-3ee060eb4f6d> type:<class 'torch.Tensor'>


Wrapped tensor with id:<UID:2582de0b-5dc8-4725-a264-3ee060eb4f6d>
Underlying Wrapper Type:syft.lib.torch.uppercase_tensor.TorchTensorWrapper


assigning id:<UID:677035f5-2f29-432f-9b43-3568cdad3669> type:<class 'torch.Tensor'>


In [23]:
x.id

<UID:e58789ca-0236-4921-99db-57dcac9a8729>

In [24]:
xp = x.send(alice_client)

the message is for me!!!


In [25]:
xp.id_at_location

<UID:e58789ca-0236-4921-99db-57dcac9a8729>

In [26]:
y = xp + xp

the message is for me!!!
torch.Tensor.__add__
(<syft.ast.klass.torch.TensorPointer object at 0x7f9740bdc760>,)
{}
<UID:fdea1f39-bd32-44b8-920f-ab1ed734b9bf>


In [27]:
y.get()

the old_message is for me!!!


<Storable:tensor([2, 4, 6, 8])>

In [28]:
alice.store

{<UID:e58789ca-0236-4921-99db-57dcac9a8729>: <Storable:tensor([1, 2, 3, 4])>}

In [29]:
del xp

Deleted:<syft.ast.klass.torch.TensorPointer object at 0x7f9740bdc760>
the old_message is for me!!!


In [30]:
alice.store

{}

In [31]:
ptr = alice_client.torch.zeros(10,10)

call torch.zeros on client <VirtualMachineClient id:alice>
the message is for me!!!


In [1]:
import numpy as np
import syft as sy

assigning id:<UID:a27d1dde-3316-4583-b8f7-072d0cd3a4ed> type:<class 'torch.Tensor'>


In [2]:
x = np.array([1,2,3,4]).send(alice_client)

assigning id:<UID:28ddbbe4-c5bb-4ca8-b30a-976dedae0713> type:<class 'syft.lib.generic.ObjectConstructor.install_id_attribute.<locals>.OriginalConstructorSubclass'>


NameError: name 'alice_client' is not defined

In [ ]:
x.get()

In [5]:
x = np.array([1,2,3,4])

assigning id:<UID:113a40ae-be5b-4062-ae29-6506d8f2b9df> type:<class 'syft.lib.generic.ObjectConstructor.install_id_attribute.<locals>.OriginalConstructorSubclass'>


In [6]:
x.serialize()

Wrapped tensor with id:<UID:113a40ae-be5b-4062-ae29-6506d8f2b9df>
Underlying Wrapper Type:syft.lib.numpy.tensor.NumpyTensorWrapper


id {
  value: "\021:@\256\276[@b\256)e\006\330\362\271\337"
}
obj_type: "syft.lib.numpy.tensor.NumpyTensorWrapper"
data {
  type_url: "type.googleapis.com/syft.lib.numpy.TensorProto"
  value: "\n\004\001\002\003\004\020\r:\300\001\000\000\000\000\000\000\000\320\026J\274\252\373\007\000\300\023\000\027\300\272\177\000\000p]\007\360\272\177\000\000\260\240\017\300\272\177\000\000\360}\017\300\272\177\000\000\360s\017\300\272\177\000\000\260]\007\360\272\177\000\000\360\343\017\300\272\177\000\0000\356\017\300\272\177\000\000p\'C\220\272\177\000\0000\341\017\300\272\177\000\000\260\357\017\300\272\177\000\000\360\267\036\240\272\177\000\0000\367!\240\272\177\000\0000u\003\340\272\177\000\000\260\371!\240\272\177\000\000\360\371!\240\272\177\000\0000\372!\240\272\177\000\0000\373!\240\272\177\000\000\260\320!\300\272\177\000\000\360\232\004\360\272\177\000\000p\325!\300\272\177\000\0000\036\003\360\272\177\000\000"
}

In [ ]:
from sqlalchemy import create_engine

engine = create_engine('sqlite:///:memory:', echo=True)

In [ ]:
from sqlalchemy.orm import sessionmaker

Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
from sqlalchemy.ext.declarative import declarative_base

Base = declarative_base()

In [ ]:
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    password = Column(String)

    def __repr__(self):
        return f'User {self.name}'

In [ ]:
Base.metadata.create_all(engine)

In [ ]:
user = User(name='John Snow', password='johnspassword')
session.add(user)

print(user.id)  # None

In [ ]:
session.commit()

In [ ]:
query = session.query(User).filter_by(name='John Snow')

In [ ]:
query.count()